<a href="https://colab.research.google.com/github/kk25gb/AIdea-project-file/blob/Siamese-network/Siamese.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [29]:
import json
import csv
from keras import layers,Input
from keras.models import Model
from urllib.request import urlopen
from sklearn.cluster import KMeans
import pandas as pd
import numpy as np

# Stage 1 資料前處理

## 讀入向量化資料


In [30]:
url = 'https://raw.githubusercontent.com/kk25gb/AIdea-project-file/main/stage1/vectorize_data.json'
response = urlopen(url)
data = json.loads(response.read())
df = pd.DataFrame(data).T

In [31]:
len(data['1'])

764

## 讀入 True label csv

In [32]:
url = 'https://raw.githubusercontent.com/kk25gb/AIdea-project-file/main/stage1/TrainLabel.csv'
response = urlopen(url)
lines = [l.decode('utf-8') for l in response.readlines()]
true_labels = list(csv.reader(lines))
del true_labels[0]

In [33]:
len(true_labels)

1381

## 建立 False label 資料

In [34]:
import random

input_keys = list(data.keys())
false_labels = []

while len(false_labels) < len(true_labels):
  temp = [random.choice(input_keys),random.choice(input_keys)]
  if temp not in true_labels:
    false_labels.append(temp)

In [35]:
len(false_labels)

1381

## 產生所需資料格式

In [36]:
def data_generator(input,Label=0):
  return [[[data[row[0]],data[row[1]]],Label] for row in input]

In [37]:
# 標籤為True的向量資料對
true_list = data_generator(true_labels,1)

In [38]:
# 標籤為False的向量資料對
false_list = data_generator(false_labels,0)

In [39]:
df_true = pd.DataFrame(true_list)
df_true.head()

,0,1
0,"[[0, 0, 0, 2, 4, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,...",1
1,"[[0, 0, 0, 2, 4, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,...",1
2,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,...",1
3,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",1
4,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",1


In [40]:
df_false = pd.DataFrame(false_list)
df_false.head()

,0,1
0,"[[0, 0, 0, 2, 4, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,...",0
1,"[[0, 0, 2, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0,...",0
2,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",0
3,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 10, 2...",0
4,"[[0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",0


## 混合 Ture / False 標籤的資料

In [41]:
train_list = true_list+false_list

In [42]:
len(train_list)

2762

In [43]:
df_train = pd.DataFrame(train_list)
df_train.head()

,0,1
0,"[[0, 0, 0, 2, 4, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,...",1
1,"[[0, 0, 0, 2, 4, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,...",1
2,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,...",1
3,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",1
4,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",1


In [44]:
from sklearn.utils import shuffle
df_train = shuffle(df_train)

In [45]:
df_train.head()

,0,1
1660,"[[0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 2,...",0
1276,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",1
222,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 6, 4,...",1
1462,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",0
393,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",1


In [46]:
X = df_train[0]
Y = df_train[1]

## 區分訓練集與驗證集

In [47]:
# from sklearn.model_selection import train_test_split
# x_train, x_vaild, y_train, y_vaild = train_test_split(X, Y, test_size=.25)

In [48]:
# len(x_train)

In [49]:
# len(x_vaild)

### 轉為Numpy陣列

In [50]:
left_x = np.array([row[0] for row in X])
right_x = np.array([row[1] for row in X])
y_train = np.array(Y)

In [51]:
pd.DataFrame(left_x_train).head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,724,725,726,727,728,729,730,731,732,733,734,735,736,737,738,739,740,741,742,743,744,745,746,747,748,749,750,751,752,753,754,755,756,757,758,759,760,761,762,763
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,10,0,0,2,2,0,0,0,0,0,3,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,5,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [52]:
pd.DataFrame(right_x_train).head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,724,725,726,727,728,729,730,731,732,733,734,735,736,737,738,739,740,741,742,743,744,745,746,747,748,749,750,751,752,753,754,755,756,757,758,759,760,761,762,763
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,10,2,0,2,2,0,0,0,0,0,3,0,0,2,0,0,2,0,0,0,0,0,0,0,0,0,5,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,4,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


# 建立神經網路模型

參考資料：
* [Image similarity estimation using a Siamese Network with a triplet loss](https://keras.io/examples/vision/siamese_network/)
* [Siamese Network — Unofficial Keras Implementation](https://github.com/gchoi/face-recognition-using-siamese-network)
* [Text-Similarity-Using-Siamese-Deep-Neural-Network](https://github.com/AnjaliDharmik/Text-Similarity-Using-Siamese-Deep-Neural-Network)

## 建構原理
藉由同一個層來一起進行萃取兩篇文章的相似度特徵  
因此需共享權重來訓練神經網路

In [53]:
lstm = layers.LSTM(382) # 共享權重的神經層

left_input = Input(shape=(None,),name='left_input')
# print(left_input.shape)
left_embedded_text = layers.Embedding(2071,764)(left_input)
left_output = lstm(left_embedded_text)
# print(left_output.shape)

right_input = Input(shape=(None,),name='right_input')
# print(right_input.shape)
right_embedded_text = layers.Embedding(2071,764)(right_input)
right_output = lstm(right_embedded_text)
# print(right_output.shape)

merged = layers.concatenate([left_output,right_output],axis=-1) # 將向量串接
# print(merged.shape)
narrow_down_1 = layers.Dense(382,activation='relu')(merged)
narrow_down_2 = layers.Dense(191,activation='relu')(narrow_down_1)
narrow_down_3 = layers.Dense(96,activation='relu')(narrow_down_2)
narrow_down_4 = layers.Dense(48,activation='relu')(narrow_down_3)
predictions = layers.Dense(1,activation='sigmoid')(narrow_down_4)
model = Model([left_input,right_input],predictions)

In [54]:
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 left_input (InputLayer)        [(None, None)]       0           []                               
                                                                                                  
 right_input (InputLayer)       [(None, None)]       0           []                               
                                                                                                  
 embedding_2 (Embedding)        (None, None, 764)    1582244     ['left_input[0][0]']             
                                                                                                  
 embedding_3 (Embedding)        (None, None, 764)    1582244     ['right_input[0][0]']            
                                                                                            

In [55]:
model = Model([left_input,right_input],predictions)
model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['acc'])

In [56]:
# 訓練方法2；使用dict將送入資料進行訓練，鍵為Input層的名稱，值為Numpy的值
model.fit({'left_input':left_x,'right_input':right_x},y_train,epochs=10,batch_size=128,validation_split=0.25)

Epoch 1/10
17/17 [==============================] - 30s 2s/step - loss: 0.6981 - acc: 0.4882 - val_loss: 0.6932 - val_acc: 0.5065
Epoch 2/10
17/17 [==============================] - 25s 1s/step - loss: 0.6938 - acc: 0.4925 - val_loss: 0.7094 - val_acc: 0.4935
Epoch 3/10
17/17 [==============================] - 25s 1s/step - loss: 0.6951 - acc: 0.4964 - val_loss: 0.6931 - val_acc: 0.5065
Epoch 4/10
17/17 [==============================] - 25s 1s/step - loss: 0.6929 - acc: 0.5176 - val_loss: 0.9515 - val_acc: 0.4935
Epoch 5/10
17/17 [==============================] - 25s 1s/step - loss: 0.7121 - acc: 0.5060 - val_loss: 0.6974 - val_acc: 0.4935
Epoch 6/10
17/17 [==============================] - 25s 1s/step - loss: 0.6933 - acc: 0.5085 - val_loss: 0.6937 - val_acc: 0.5065
Epoch 7/10
17/17 [==============================] - 25s 1s/step - loss: 0.6937 - acc: 0.4896 - val_loss: 0.6942 - val_acc: 0.4935
Epoch 8/10
17/17 [==============================] - 25s 1s/step - loss: 0.6943 - acc: 0.49

# 載入Stage 2 比對資料


In [57]:
url = 'https://raw.githubusercontent.com/kk25gb/AIdea-project-file/main/stage2/vectorize_data.json'
response = urlopen(url)
predict_data = json.loads(response.read())
df = pd.DataFrame(predict_data).T

In [62]:
known_assay = list(data.values())
compared_assay = list(predict_data.values())

In [68]:
len(known_assay[0])

764

In [ ]:
array = np.array(known_assay[0])
np.reshape(array, array.shape + (1,))

In [69]:
len(compared_assay[0])

764

In [67]:
model.predict({'left_input':np.array(known_assay[0]),'right_input':np.array(compared_assay[0])})

ValueError: ignored

In [58]:
results = {}
for predict_row in predict_data:
  for known_row in data:
    temp = []
    if model.predict({'left_input':predict_row,'right_input':known_row}) == 1:
      temp.append(known_row)
    results[predict_row] = known_row

ValueError: ignored

In [56]:
model.predict({'left_input':left_x,'right_input':right_x})